In [1]:
import numpy as np
import pandas as pd
import csv
import json
import math

# Suppressing scientific notation in pandas
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
path_to_trip2017 = r'E:\Demo\NHTS2017_csv\trippub.csv'
trip17 = pd.read_csv(path_to_trip2017)

Please read section *7.11 Weighting the Data* in [2017 Users' Guide](https://nhts.ornl.gov/assets/NHTS2017_UsersGuide_04232019_1.pdf) first.

**Note:** The process of applying the weights is specific to Python. Check the SPSS and/or Stata documentation for instructions on how to weight and then summarize weighted data.

### Vehicle Trips (VT) by Trip Purpose Summary in Weekdays

For filter condition for VT and VMT, refer to sections *7.5 Vehicle Trips* and *7.6 Vehicle Miles of Travel (VMT)* 
in the 2017 Users' Guide.

In [47]:
# Select VT trips for VT calculation:
#    (TRPTRANS in pov17) AND (DVRV_FLG == 1)
pov17 = [3, 4, 5, 6, 8, 9, 18]
weekdays = [2, 3, 4, 5, 6]

# filter trip data:
#    1. VT selection
#    2. Weekday selection
trip17_vt = trip17[(trip17['TRPTRANS'].isin(pov17)) & 
                   (trip17['DRVR_FLG'] == 1) &
                  (trip17['TRAVDAY'].isin(weekdays))].copy()

#### One-way table: VT by trip purpose summary (weighted sum and pct) 

In [43]:
# check for sample size (unweighted VT by WHYTRP1S)
# always always check sample size - anything below 30 is considered small sample size

trip17_vt.groupby(['WHYTRP1S']).size()

WHYTRP1S
1     167534
10     86356
20     10791
30     11144
40    105887
50     38880
70     39775
80     33693
97      8512
dtype: int64

In [48]:
# VT by trip purpose summary (weighted VT by WHYTRP1S and PCT)

# sum the ***trip weight*** based on categories in WHYTRP1S
tb1 = trip17_vt.groupby(['WHYTRP1S'], as_index=0).agg({'WTTRDFIN':'sum'})
# add column percent
tb1['pct'] = tb1.apply(lambda x: x['WTTRDFIN']/tb1['WTTRDFIN'].sum(), axis=1)
# table
tb1

,WHYTRP1S,WTTRDFIN,pct
0,1,56773803926.193,0.339
1,10,32984557026.416,0.197
2,20,4082993913.223,0.024
3,30,3111807572.194,0.019
4,40,30030044513.503,0.179
5,50,11915530691.404,0.071
6,70,15900921288.691,0.095
7,80,10365304374.367,0.062
8,97,2322212901.146,0.014


In [18]:
# rename columns:
# 'WHYTRP1S' -> 'trip_purp_summary'
# 'WTTRDFIN' -> "annual_total_VT"

tb1 = tb1.rename(columns = {'WHYTRP1S':'trip_purp_summary',
                           'WTTRDFIN':'annual_total_VT'})
tb1

,trip_purp_summary,annual_total_VT,pct
0,1,56773803926.193,0.339
1,10,32984557026.416,0.197
2,20,4082993913.223,0.024
3,30,3111807572.194,0.019
4,40,30030044513.503,0.179
5,50,11915530691.404,0.071
6,70,15900921288.691,0.095
7,80,10365304374.367,0.062
8,97,2322212901.146,0.014


DE Table Reference: https://nhts.ornl.gov/tables09/ae/work/Job146738.html. The "All" column shows the annual_total_VT.

#### Two-way table: VT by trip purpose summary by weekday (weighted sum) 

In [10]:
# create cross-tabulation for weighted annual sum
# for percentages, please save (or copy paste) the output table below into Excel
# and manually compute the desired percentages.
pd.crosstab(trip17_vt['TRAVDAY'], trip17_vt['WHYTRP1S'],
            trip17_vt['WTTRDFIN'], aggfunc=sum,
            dropna=False,
            margins=True
)

WHYTRP1S,1,10,20,30,40,50,70,80,97,All
TRAVDAY,,,,,,,,,,
2,11183417328.193,6279719900.301,738014657.122,614716506.445,6192729175.945,2363516119.763,2934112615.612,1792707038.736,352610323.693,32451543665.811
3,11254512246.526,6714556629.077,848311319.994,698677659.642,5683569757.132,2123443047.902,3201510573.695,1967992754.804,515694570.763,33008268559.534
4,11688116905.686,6725758520.499,1035374508.322,670552695.003,5896917889.554,2319860921.099,3431222364.171,2017080541.280,515672232.200,34300556577.814
5,11290546801.581,6956538920.293,856774546.873,623973188.946,5791780977.482,2456577691.406,3201243089.409,2057225720.294,510684262.983,33745345199.267
6,11357210644.207,6307983056.246,604518880.912,503887522.157,6465046713.391,2652132911.233,3132832645.805,2530298319.253,427551511.506,33981462204.709
All,56773803926.193,32984557026.416,4082993913.223,3111807572.194,30030044513.503,11915530691.404,15900921288.691,10365304374.367,2322212901.146,167487176207.148


In [46]:
# create cross-tabulation for sample size
# always always check sample size - anything below 30 is considered small sample size
pd.crosstab(trip17_vt['TRAVDAY'], trip17_vt['WHYTRP1S'],
             trip17_vt['WTTRDFIN'], aggfunc=lambda x: x.count(),
            dropna=False,
            margins=True
)

WHYTRP1S,1,10,20,30,40,50,70,80,97,All
TRAVDAY,,,,,,,,,,
2,32021.000,16017.000,1767.000,2268.000,20139.000,7357.000,7116.000,5660.000,1474.000,93819.000
3,34065.000,18088.000,2450.000,2369.000,21273.000,7395.000,8111.000,6370.000,1918.000,102039.000
4,33849.000,17881.000,2621.000,2363.000,20585.000,7546.000,8269.000,6738.000,1679.000,101531.000
5,34034.000,18185.000,2303.000,2359.000,20813.000,8025.000,8346.000,6795.000,1858.000,102718.000
6,33565.000,16185.000,1650.000,1785.000,23077.000,8557.000,7933.000,8130.000,1583.000,102465.000
All,167534.000,86356.000,10791.000,11144.000,105887.000,38880.000,39775.000,33693.000,8512.000,502572.000


DE Table Reference: https://nhts.ornl.gov/tables09/ae/work/Job146736.html.